# Practical Application III: Comparing Classifiers

**Overview**: In this practical application, your goal is to compare the performance of the classifiers we encountered in this section, namely K Nearest Neighbor, Logistic Regression, Decision Trees, and Support Vector Machines.  We will utilize a dataset related to marketing bank products over the telephone.  



### Getting Started

Our dataset comes from the UCI Machine Learning repository [link](https://archive.ics.uci.edu/ml/datasets/bank+marketing).  The data is from a Portugese banking institution and is a collection of the results of multiple marketing campaigns.  We will make use of the article accompanying the dataset [here](CRISP-DM-BANK.pdf) for more information on the data and features.



### Problem 1: Understanding the Data

To gain a better understanding of the data, please read the information provided in the UCI link above, and examine the **Materials and Methods** section of the paper.  How many marketing campaigns does this data represent?

### Problem 1 Answer

The data represents 17 marketing campaigns among 79,354 contacts.

### Problem 2: Read in the Data

Use pandas to read in the dataset `bank-additional-full.csv` and assign to a meaningful variable name.

In [193]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.dummy import DummyClassifier
import time

In [199]:
df = pd.read_csv('data/bank-additional-full.csv', sep = ';')

In [200]:
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


### Problem 3: Understanding the Features


Examine the data description below, and determine if any of the features are missing values or need to be coerced to a different data type.


```
Input variables:
# bank client data:
1 - age (numeric)
2 - job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
3 - marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
4 - education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')
5 - default: has credit in default? (categorical: 'no','yes','unknown')
6 - housing: has housing loan? (categorical: 'no','yes','unknown')
7 - loan: has personal loan? (categorical: 'no','yes','unknown')
# related with the last contact of the current campaign:
8 - contact: contact communication type (categorical: 'cellular','telephone')
9 - month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
10 - day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')
11 - duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.
# other attributes:
12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
14 - previous: number of contacts performed before this campaign and for this client (numeric)
15 - poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')
# social and economic context attributes
16 - emp.var.rate: employment variation rate - quarterly indicator (numeric)
17 - cons.price.idx: consumer price index - monthly indicator (numeric)
18 - cons.conf.idx: consumer confidence index - monthly indicator (numeric)
19 - euribor3m: euribor 3 month rate - daily indicator (numeric)
20 - nr.employed: number of employees - quarterly indicator (numeric)

Output variable (desired target):
21 - y - has the client subscribed a term deposit? (binary: 'yes','no')
```



### Problem 4: Understanding the Task

After examining the description and data, your goal now is to clearly state the *Business Objective* of the task.  State the objective below.

In [201]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

## Business Objective

Given the dataset, the goal is to analyze what features of the customer contribute to a successful directed marketing campaign to improve the success rate of the business. Directed marketing campaigns, by nature, are more specific and personalized to the consumer, so figuring out what features are conducive to a subscription is very helpful.

### Problem 5: Engineering Features

Now that you understand your business objective, we will build a basic model to get started.  Before we can do this, we must work to encode the data.  Using just the bank information features, prepare the features and target column for modeling with appropriate encoding and transformations.

In [202]:
# -----------------------------
# 1. Select Features and Target
# -----------------------------

df['y'] = df['y'].map({'yes': 1, 'no': 0})
X = df.drop(columns=["y", 'duration'])

num_features = X.select_dtypes(include=['number']).columns.to_list()
cat_features = X.select_dtypes(include=['object']).columns.to_list()

y = df['y']

In [203]:
# -----------------------------
# 2. Scale and Encode Features
# -----------------------------

preprocess = ColumnTransformer([
    ("num", StandardScaler(), num_features),
    ("cat", OneHotEncoder(handle_unknown="ignore"), cat_features)
])

### Problem 6: Train/Test Split

With your data prepared, split it into a train and test set.

In [204]:
# -----------------------------
# 3. Generate Train/Test Split
# -----------------------------

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.25, shuffle=True)

### Problem 7: A Baseline Model

Before we build our first model, we want to establish a baseline.  What is the baseline performance that our classifier should aim to beat?

In [205]:
dummy_clf = DummyClassifier().fit(X_train, y_train)
baseline_score = dummy_clf.score(X_test, y_test)

In [206]:
baseline_score

0.8880256385354958

### Accuracy
- ratio of correctly predicted observations to total observations
- overall review of performance, better on balaned datasets

In [207]:
def get_accuracy(cm):
    correct = cm[0][0] + cm[1][1]
    incorrect = cm[1][0] + cm[0][1]

    acc = (correct / (correct + incorrect)) * 100
    return acc

### Precision
- ratio of correct positive predictions to total positive predictions
- of all the model's positive predictions, how many were correct

In [208]:
def get_prec(cm):
    return (cm[1][1] / (cm[0][1]  + cm[1][1] )) * 100

### Recall
- precision at which model can return relevant data
- of all the actual positive cases, how many were predicted

In [209]:
def get_recall(cm):
    return (cm[1][1] / (cm[1][0]  + cm[1][1] )) * 100

### F1
- weighted average of precision and recall

In [210]:
def get_f1(cm):
    return (2 * recall * prec) / (recall + prec)

### Problem 8: A Simple Model

Use Logistic Regression to build a basic model on your data.  

In [211]:
# -----------------------------
# 4. Creating a Model
# -----------------------------

start = time.perf_counter()

reg_pipe = Pipeline([
    ("prep", preprocess),
    ("reg", LogisticRegression())
])

reg_pipe.fit(X_train, y_train)
regression_fit_time = time.perf_counter() - start

In [212]:
regression_fit_time

0.1835795830047573

In [213]:
y_pred_test = reg_pipe.predict(X_test)
y_pred_train = reg_pipe.predict(X_train)
cm_regression_test = confusion_matrix(y_test, y_pred_test)
cm_regression_train = confusion_matrix(y_train, y_pred_train)

### Problem 9: Score the Model

What is the accuracy of your model?

In [214]:
train_reg_accuracy = get_accuracy(cm_regression_train)
test_reg_accuracy = get_accuracy(cm_regression_test)

print("Test Accuracy Logistic Regression: ", test_reg_accuracy)
print("Train Accuracy Logistic Regression: ", train_reg_accuracy)

Test Accuracy Logistic Regression:  89.9970865300573
Train Accuracy Logistic Regression:  90.03269560713476


### Problem 10: Model Comparisons

Now, we aim to compare the performance of the Logistic Regression model to our KNN algorithm, Decision Tree, and SVM models.  Using the default settings for each of the models, fit and score each.  Also, be sure to compare the fit time of each of the models.  Present your findings in a `DataFrame` similar to that below:

| Model | Train Time | Train Accuracy | Test Accuracy |
| ----- | ---------- | -------------  | -----------   |
|     |    |.     |.     |

## KNN Model

- K Nearest Neighbors
    - model simply plots all points in feature space and picks the k nearest points for each new sample for inference
    - little model training time
    - not great for hebaily imbalanced data sets

In [215]:
start = time.perf_counter()
knn_pipe = Pipeline([
    ("prep", preprocess),
    ('knn', KNeighborsClassifier())
])

knn_pipe.fit(X_train, y_train)
knn_fit_time = time.perf_counter() - start

In [216]:
y_pred_test = knn_pipe.predict(X_test)
y_pred_train = knn_pipe.predict(X_train)
cm_knn_test = confusion_matrix(y_test, y_pred_test)
cm_knn_train = confusion_matrix(y_train, y_pred_train)

In [217]:
train_knn_accuracy = get_accuracy(cm_knn_train)
test_knn_accuracy = get_accuracy(cm_knn_test)

print("Test Accuracy KNN: ", test_knn_accuracy)
print("Train Accuracy KNN: ", train_knn_accuracy)

Test Accuracy KNN:  88.96766048363601
Train Accuracy KNN:  91.39878929137937


## Decision Tree Classifier

- Decision Tree Classifier
    - model classifies batches through series of yes or no questions to make splits
    - tree grows with leaves of yes or no classifications
    - deeper trees = more accurate but may overfit

In [218]:
start = time.perf_counter()
dtc = Pipeline([
    ("prep", preprocess),
    ('decision_tree', DecisionTreeClassifier())
])

dtc.fit(X_train, y_train)
dtc_fit_time = time.perf_counter() - start

In [219]:
y_pred_test = dtc.predict(X_test)
y_pred_train = dtc.predict(X_train)
cm_dtc_test = confusion_matrix(y_test, y_pred_test)
cm_dtc_train = confusion_matrix(y_train, y_pred_train)

In [220]:
train_dtc_accuracy = get_accuracy(cm_dtc_train)
test_dtc_accuracy = get_accuracy(cm_dtc_test)

print("Test Accuracy Decision Trees: ", test_dtc_accuracy)
print("Train Accuracy Decision Trees: ", train_dtc_accuracy)

Test Accuracy Decision Trees:  84.31582014178886
Train Accuracy Decision Trees:  99.54355637564339


## Support Vector Classifier

In [221]:
start = time.perf_counter()
svc_pipe = Pipeline([
    ("prep", preprocess),
    ('svc', SVC())
])

svc_pipe.fit(X_train, y_train)
svc_fit_time = time.perf_counter() - start

In [222]:
y_pred_test = svc_pipe.predict(X_test)
y_pred_train = svc_pipe.predict(X_train)
cm_svc_test = confusion_matrix(y_test, y_pred_test)
cm_svc_train = confusion_matrix(y_train, y_pred_train)

In [223]:
train_svc_accuracy = get_accuracy(cm_svc_train)
test_svc_accuracy = get_accuracy(cm_svc_test)

print("Test Accuracy SVC: ", test_svc_accuracy)
print("Train Accuracy SVC: ", train_svc_accuracy)

Test Accuracy SVC:  90.00679809653298
Train Accuracy SVC:  90.44381858793822


In [224]:
data = {
    "Model": ["Logistic Regression", "K Nearest Neighbors", "Decision Tree Classifier",
                 "Support Vector Classifier"],
    "Train Time": [regression_fit_time, knn_fit_time, dtc_fit_time, svc_fit_time],
    "Train Accuracy": [train_reg_accuracy, train_knn_accuracy, train_dtc_accuracy, train_svc_accuracy],
    "Test Accuracy": [test_reg_accuracy, test_knn_accuracy, test_dtc_accuracy, test_svc_accuracy]
}

df = pd.DataFrame.from_dict(data)
df

,Model,Train Time,Train Accuracy,Test Accuracy
0,Logistic Regression,0.183580,90.032696,89.997087
1,K Nearest Neighbors,0.082431,91.398789,88.967660
2,Decision Tree Classifier,0.222151,99.543556,84.315820
3,Support Vector Classifier,51.913085,90.443819,90.006798


### Problem 11: Improving the Model

Now that we have some basic models on the board, we want to try to improve these.  Below, we list a few things to explore in this pursuit.


- Hyperparameter tuning and grid search.  All of our models have additional hyperparameters to tune and explore.  For example the number of neighbors in KNN or the maximum depth of a Decision Tree.  
- Adjust your performance metric

## Logistic Regression Hyperparameters

- C value
- penalty type

In [225]:
params = {
         'reg__C': [0.01, 0.1],
         'reg__penalty': ['l2'],
         }

regression_grid = GridSearchCV(
    estimator=reg_pipe,
    param_grid=params,
    scoring='accuracy'
)

regression_grid.fit(X_train, y_train)
best_regression_model = regression_grid.best_estimator_

y_pred_test = best_regression_model.predict(X_test)
y_pred_train = best_regression_model.predict(X_train)
cm_regression_test = confusion_matrix(y_test, y_pred_test)
cm_regression_train = confusion_matrix(y_train, y_pred_train)

train_reg_accuracy = get_accuracy(cm_regression_train)
test_reg_accuracy = get_accuracy(cm_regression_test)

print("Test Accuracy Logistic Regression: ", test_reg_accuracy)
print("Train Accuracy Logistic Regression: ", train_reg_accuracy)

Test Accuracy Logistic Regression:  90.04564436243567
Train Accuracy Logistic Regression:  90.06183030656179


In [229]:
reg_prec = get_prec(cm_regression_test)
reg_recall = get_recall(cm_regression_test)
reg_f1 = get_f1(cm_regression_test)

print("Test Precision Logistic Regression: ", reg_prec)
print("Test Recall Logistic Regression: ", reg_recall)
print("Test F1 Logistic Regression: ", reg_f1)

Test Precision Logistic Regression:  67.77777777777779
Test Recall Logistic Regression:  21.16218560277537
Test F1 Logistic Regression:  10.642040457343887


## KNN Hyperparameters

- n_nieghbors
    - during inference, how many neighbors are accounted
- weights
    - how is each neighbor weighted, are closer neighbors given a greater preference
- p
    - how the distance from inference point is calculated

In [226]:
params2 = {"knn__p": [1,2],
          'knn__weights': ['uniform', 'distance'],
          'knn__n_neighbors': [5, 10, 15]
          }

knn_grid = GridSearchCV(
    estimator=knn_pipe,
    param_grid=params2,
    scoring='accuracy'
)

knn_grid.fit(X_train, y_train)
best_knn_model = knn_grid.best_estimator_

y_pred_test = best_knn_model.predict(X_test)
y_pred_train = best_knn_model.predict(X_train)
cm_knn_test = confusion_matrix(y_test, y_pred_test)
cm_knn_train = confusion_matrix(y_train, y_pred_train)

train_knn_accuracy = get_accuracy(cm_knn_train)
test_knn_accuracy = get_accuracy(cm_knn_test)

print("Test Accuracy KNN: ", test_knn_accuracy)
print("Train Accuracy KNN: ", train_knn_accuracy)

Test Accuracy KNN:  89.7154511022628
Train Accuracy KNN:  90.43410702146257


In [234]:
knn_prec = get_prec(cm_knn_test)
knn_recall = get_recall(cm_knn_test)
knn_f1 = get_f1(cm_knn_test)

print("Test Precision KNN: ", knn_prec)
print("Test Recall KNN: ", knn_recall)
print("Test F1 KNN: ", knn_f1)

Test Precision KNN:  59.28853754940712
Test Recall KNN:  26.019080659150045
Test F1 KNN:  10.642040457343887


## Decision Tree Classifier Hyperparameters

- min samples split
- criterion
- max depth
- min samples leaf

In [236]:
params3 = {
    "decision_tree__max_depth": [5, 10, 20],
    "decision_tree__min_samples_split": [2, 5, 10, 20],
    "decision_tree__min_samples_leaf": [1, 2, 5, 10],
    "decision_tree__criterion": ["gini", "entropy"]
}

dtc_grid = GridSearchCV(
    estimator=dtc,
    param_grid=params3,
    scoring='accuracy'
)

dtc_grid.fit(X_train, y_train)
best_dtc_model = dtc_grid.best_estimator_

y_pred_test = best_dtc_model.predict(X_test)
y_pred_train = best_dtc_model.predict(X_train)
cm_dtc_test = confusion_matrix(y_test, y_pred_test)
cm_dtc_train = confusion_matrix(y_train, y_pred_train)

train_dtc_accuracy = get_accuracy(cm_dtc_train)
test_dtc_accuracy = get_accuracy(cm_dtc_test)

print("Test Accuracy Decision Trees: ", test_dtc_accuracy)
print("Train Accuracy Decision Trees: ", train_dtc_accuracy)

Test Accuracy Decision Trees:  90.00679809653298
Train Accuracy Decision Trees:  90.34022854553106


In [237]:
dtc_prec = get_prec(cm_dtc_test)
dtc_recall = get_recall(cm_dtc_test)
dtc_f1 = get_f1(cm_dtc_test)

print("Test Precision Decision Tree Classifier: ", dtc_prec)
print("Test Recall Decision Tree Classifier: ", dtc_recall)
print("Test F1 Decision Tree Classifier: ", dtc_f1)

Test Precision Decision Tree Classifier:  63.47826086956522
Test Recall Decision Tree Classifier:  25.32523850823938
Test F1 Decision Tree Classifier:  10.642040457343887


## Support Vector Classifier Parameters

- kernel
    - type of kernel function applied 
- gamma
- C

In [228]:
params4 = {
    "svc__kernel": ["rbf", "poly", "linear"],
    "svc__gamma": [0.01, 0.1],
}

svc_grid = GridSearchCV(
    estimator=svc_pipe,
    param_grid=params4,
    scoring='accuracy'
)

svc_grid.fit(X_train, y_train)
best_svc_model = svc_grid.best_estimator_


y_pred_test = best_svc_model.predict(X_test)
y_pred_train = best_svc_model.predict(X_train)
cm_svc_test = confusion_matrix(y_test, y_pred_test)
cm_svc_train = confusion_matrix(y_train, y_pred_train)

train_svc_accuracy = get_accuracy(cm_svc_train)
test_svc_accuracy = get_accuracy(cm_svc_test)

print("Test Accuracy SVC: ", test_svc_accuracy)
print("Train Accuracy SVC: ", train_svc_accuracy)

Test Accuracy SVC:  90.01650966300865
Train Accuracy SVC:  91.01680101000291


In [238]:
svc_prec = get_prec(cm_svc_test)
svc_recall = get_recall(cm_svc_test)
svc_f1 = get_f1(cm_svc_test)

print("Test Precision Decision Tree Classifier: ", svc_prec)
print("Test Recall Decision Tree Classifier: ", svc_recall)
print("Test F1 Decision Tree Classifier: ", svc_f1)

Test Precision Decision Tree Classifier:  63.61655773420479
Test Recall Decision Tree Classifier:  25.32523850823938
Test F1 Decision Tree Classifier:  10.642040457343887


##### Questions